In [76]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [85]:
class Bottleneck(nn.Module):

    def __init__(self, in_channels, reduced_channels, out_channels, stride=1, starting=False):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, reduced_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(reduced_channels),
            nn.ReLU(inplace=True)
        ) 
        self.conv2 = nn.Sequential(
            nn.Conv2d(reduced_channels, reduced_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(reduced_channels),
            nn.ReLU(inplace=True)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(reduced_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
        )
        self.shortcut = nn.Sequential()

        if in_channels != out_channels:
            self.shortcut.add_module(
                'conv',
                nn.Conv2d(
                    in_channels,
                    out_channels,
                    kernel_size=1,
                    stride=stride,  # downsample
                    padding=0,
                    bias=False))
            self.shortcut.add_module('bn', nn.BatchNorm2d(out_channels))  # BN
        self.final_relu = nn.ReLU(inplace=True)


    def forward(self, x):
        out = self.conv1(x)
        print(f"size of self.conv1: {out.size()}")
        out = self.conv2(out)
        print(f"size of self.conv2: {out.size()}")
        # print(f"size of self.shortcut(x): {self.shortcut(x).size()}")
        out = self.conv3(out) + self.shortcut(x)
        out = self.final_relu(out)
        print(f"size of output of bottleneck: {out.size()}")
        return out


In [89]:
class ResNet50(nn.Module):

    def __init__(self, num_classes=1000):
        super(ResNet50, self).__init__()
        self.num_blocks = [3,4,6,3]
        # Conv1 layer: 224*224*3 -> 112*112*64
        # self.conv1 = nn.Sequential(
        #     nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=4),
        #     nn.BatchNorm2d(num_features=64),
        #     nn.ReLU(inplace=True),
        #     #nn.MaxPool2d(kernel_size=3, stride=2)
        # )
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=4)
        self.bn1 = nn.BatchNorm2d(num_features=64)
        self.rl1 = nn.ReLU(inplace=True)
        self.max1 = nn.MaxPool2d(kernel_size=3, stride=2)

        #self.conv2 = self._make_layer(in_channels=64, reduced_channels=64, out_channels=256, num_blocks=self.num_blocks[0], stride=1)
        #self.conv3 = self._make_layer(in_channels=256, reduced_channels=128, out_channels=512, num_blocks=self.num_blocks[1], stride=2)
        #self.conv4 = self._make_layer(in_channels=512, reduced_channels=256, out_channels=1024, num_blocks=self.num_blocks[2], stride=2)
        #self.conv5 = self._make_layer(in_channels=1024, reduced_channels=512, out_channels=2048, num_blocks=self.num_blocks[3], stride=2)
        
        self.avg = nn.AvgPool2d(kernel_size=7, stride=1)
        # FC layer, after applied 'avg pooling'
        self.fc1 = nn.Sequential(
            nn.Linear(in_features=2048, out_features=1000)
        )

    def _make_layer(self, in_channels, reduced_channels, out_channels, num_blocks, stride=1):
        pass
        layers = []
        strides = [stride] + [1]*(num_blocks-1)
        print("num blocks: ", num_blocks)

        for i, s in enumerate(strides):
            print(f"i: {i}")
            bottleneck_layer = Bottleneck(in_channels, reduced_channels, out_channels, s)
            layers.append(bottleneck_layer)
            in_channels = in_channels *4
        print("Done")
        
        return nn.Sequential(*layers)

    def forward(self, x):
        print("input size: ", x.size())
        print("Start foward pass")
        out = self.max1(self.rl1(self.bn1(self.conv1(x))))
        print("ResNet50 self.conv1 done, size: ", self.conv1(x).size())
        out = self.conv2(out)
        print("ResNet50 self.conv2 done")
        out = self.conv3(out)
        # print("ResNet50 self.conv3 done")
        # out = self.conv4(out)
        # print("ResNet50 self.conv4 done")
        # out = self.conv5(out)
        out = out.view(out.size(0), -1)
        out = nn.avg(out)
        out = self.fc1(out)
        return out

In [90]:
from torchsummary import summary

resnet = ResNet50()
summary(resnet, input_size=(3,28,28))

input size:  torch.Size([2, 3, 28, 28])
Start foward pass


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same